<a href="https://colab.research.google.com/github/akritidu/Multi-label-Classification-in-the-Home-Improvement/blob/main/LSTM_Bi_LSTM_GRU_Bi_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Uploading and Cleaning 

In [1]:

from numpy import array
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, GRU, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D

import tensorflow as tf
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [ ]:
# df2.to_csv('2023-08-08-clean_file.csv', index=False)

In [2]:
df = pd.read_csv('2023-08-08-clean_file.csv') #actual

In [3]:
train_df = df

In [13]:
train_df

,contexts,ham,phishing
0,google window google bypjtqaxxnab getei functi...,1,0
1,function envflush function window require requ...,1,0
2,timing timing timing tick function label time ...,1,0
3,yahoo deutschland roundtrip rttop number date ...,1,0
4,html overflow-y auto body font arial text-alig...,1,0
5,wikipedia unicode-bidi embed white-space nowra...,1,0
6,dtfirstbyte date meta http-equiv refresh conte...,1,0
7,blogger erstellen kostenloses blog html body t...,1,0
8,twitter document domain twitter copy twttr app...,1,0
9,body margin pad font-size line-height font-fam...,1,0


In [14]:
categories = df[['ham', 'phishing']]
categories.head()

,ham,phishing
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

#Lemmatize Words

def fetch_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()

#cleaning the data now 

regex = [
    r'<[^>]+>', #HTML tags
    r'@(\w+)', # @-mentions
    r"#(\w+)", # hashtags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'[^0-9a-z #+_\\r\\n\\t]', #BAD SYMBOLS
]

REPLACE_URLS = re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
REPLACE_HASH = re.compile(r'#(\w+)')
REPLACE_AT = re.compile(r'@(\w+)')
REPLACE_HTML_TAGS = re.compile(r'<[^>]+>')
REPLACE_DIGITS = re.compile(r'\d+')
#REPLACE_BY = re.compile(r"[/(){}\[\]\|,;.:?\-\'\"$]")
REPLACE_BY = re.compile(r"[^a-z0-9\-]")

STOPWORDS = set(stopwords.words('english'))

#tokens_re = re.compile(r'('+'|'.join(regex)+')', re.VERBOSE | re.IGNORECASE)

# sentences = [] #for Word2Vec model

def clean_text(text):
    text = text.lower()
    text = REPLACE_HTML_TAGS.sub(' ', text)
    text = REPLACE_URLS.sub('', text)
    text = REPLACE_HASH.sub('', text)
    text = REPLACE_AT.sub('', text)
    text = REPLACE_DIGITS.sub(' ', text)
    text = REPLACE_BY.sub(' ', text)
    
    
    text = " ".join(lemmatizer.lemmatize(word.strip(), fetch_pos_tag(pos_tag([word.strip()])[0][1])) for word in text.split() if word not in STOPWORDS and len(word)>3)
    
    #sentences.append(text.split())
    return text



In [ ]:
# train_df['contexts'] = train_df['contexts'].apply(clean_text) 

In [16]:
## for single output layer
X  = df['contexts'] 
y = categories.values

In [18]:
#Prueba 2 para evaluar 
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score

# # comento el 16 de goto de 2023
# Tokenizar los datos de texto
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.word_index) + 1
# Ajustar las secuencias a una longitud fija
maxlen = 200
X = pad_sequences(X, padding='post', maxlen=maxlen)
# # cierro comento el 16 de goto de 2023


# tokenizer = Tokenizer(num_words=3000)
# tokenizer.fit_on_texts(X_train)

# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

# vocab_size = len(tokenizer.word_index) + 1

# maxlen = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [19]:
from numpy import array
from numpy import asarray
from numpy import zeros


embeddings_dictionary = dict()

glove_file = open("../Datasets/glove.6B.100d.txt", encoding="utf-8")
# open("glove.6B.50d.txt", 'r', encoding="utf-8") como f:

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

# **BI-GRU 64**

In [20]:
import tensorflow as tf
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D


In [21]:
vocab_size

173245

In [22]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import tensorflow as tf
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
model = SpatialDropout1D(0.2)(embedding_layer)
model = Bidirectional(GRU(64,return_sequences=True,dropout=0.1,recurrent_dropout=0.1 ))(model)
model = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
avg_pool = GlobalAveragePooling1D()(model)
max_pool = GlobalMaxPooling1D()(model)
model = concatenate([avg_pool, max_pool])
dense_layer1 = Dense(2, activation="sigmoid")(model)
Bi_GRU_glove = Model(deep_inputs, dense_layer1)
Bi_GRU_glove.compile(loss='binary_crossentropy',optimizer='adam',metrics=[tf.keras.metrics.AUC()])

In [23]:
# Crear una instancia de kfolds con n=5
kf = KFold(n_splits=5, shuffle=True)

# Lista para almacenar los resultados de precisión
accuracy_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    Bi_GRU_glove.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    y_pred = np.argmax(Bi_GRU_glove.predict(X_test), axis=-1)
    accuracy = accuracy_score(np.argmax(y_test, axis=-1), y_pred)
#     print()
    print(accuracy)
    accuracy_scores.append(accuracy)
    

# Calcular la precisión media en todos los folds
mean_accuracy = np.mean(accuracy_scores)
print('Mean Accuracy over KFold = 5:', mean_accuracy)

0.9513568868407578
0.9549180327868853
0.9779713114754098
0.9815573770491803
0.983094262295082
Mean Accuracy over KFold = 5: 0.969779574089463


## FIN

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import tensorflow as tf
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
model = SpatialDropout1D(0.2)(embedding_layer)
model = Bidirectional(GRU(64,return_sequences=True,dropout=0.1,recurrent_dropout=0.1 ))(model)
model = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
avg_pool = GlobalAveragePooling1D()(model)
max_pool = GlobalMaxPooling1D()(model)
model = concatenate([avg_pool, max_pool])
dense_layer1 = Dense(2, activation="sigmoid")(model)
Bi_GRU_glove = Model(deep_inputs, dense_layer1)
Bi_GRU_glove.compile(loss='binary_crossentropy',optimizer='adam',metrics=[tf.keras.metrics.AUC()])

In [ ]:
# Crear una instancia de kfolds con n=5
kf = KFold(n_splits=5, shuffle=True)

# Lista para almacenar los resultados de precisión
accuracy_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    Bi_GRU_glove.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    y_pred = np.argmax(Bi_GRU_glove.predict(X_test), axis=-1)
    accuracy = accuracy_score(np.argmax(y_test, axis=-1), y_pred)
#     print()
    print(accuracy)
    accuracy_scores.append(accuracy)
    

# Calcular la precisión media en todos los folds
mean_accuracy = np.mean(accuracy_scores)
print('Mean Accuracy over KFold = 5:', mean_accuracy)